<a href="https://colab.research.google.com/github/Yoonsun222/CapstoneDesign/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df1  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.01.31-02.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df2  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.07-02.13.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df3  = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.14-02.20.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df4 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.21-02.27.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
df5 = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/CHILD_AIR_2022.02.28-03.06.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
weather_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/기상청데이터.csv", encoding = 'euc-kr', parse_dates=['등록일자'])
dust_df = pd.read_csv("/content/drive/MyDrive/캡스톤디자인/서울시미세먼지평균.csv",encoding = 'euc-kr', parse_dates=['등록일자'] )

df = pd.concat([df1,df2,df3,df4,df5],axis=0, join='inner')



In [6]:
df['등록일자'] = df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

df

,시리얼,관측시각,온도(℃),습도(%),미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자
0,V10G1802191,2.021050e+11,20,49,5,4,99,405,2022-01-31 00:00
1,V10G1802099,2.021050e+11,21,46,4,3,106,447,2022-01-31 00:00
2,V10G1802225,2.021050e+11,21,49,4,3,27,461,2022-01-31 00:00
3,V10G1802415,2.021050e+11,17,49,4,3,581,383,2022-01-31 00:00
4,V10G1802677,2.021050e+11,21,46,5,4,7,570,2022-01-31 00:00
...,...,...,...,...,...,...,...,...,...
642691,V10G1803626,2.021050e+11,22,49,3,2,143,703,2022-03-06 23:00
642692,V10G1802586,2.021050e+11,21,44,3,3,10,372,2022-03-06 23:00
642693,V10G1802051,2.021050e+11,21,49,4,3,25,69,2022-03-06 23:00
642694,V10G1802859,2.021050e+11,21,61,3,3,4,383,2022-03-06 23:00


In [7]:
weather_df['등록일자'] = weather_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

weather_df

,등록일자,기온(°C),풍속(m/s),습도(%),일조(hr),전운량(10분위)
0,2022-01-31 01:00,-4.6,2.4,51,0.0,0
1,2022-01-31 02:00,-5.0,2.3,52,0.0,0
2,2022-01-31 03:00,-5.3,1.6,55,0.0,0
3,2022-01-31 04:00,-5.8,1.0,55,0.0,0
4,2022-01-31 05:00,-6.1,1.9,58,0.0,0
...,...,...,...,...,...,...
811,2022-03-05 20:00,1.5,3.9,37,0.0,0
812,2022-03-05 21:00,0.7,3.3,40,0.0,0
813,2022-03-05 22:00,0.0,2.9,40,0.0,0
814,2022-03-05 23:00,-0.3,3.0,41,0.0,0


In [8]:
dust_df['등록일자'] = dust_df["등록일자"].dt.strftime("%Y-%m-%d %H:00")

dust_df

,등록일자,1시간평균 미세먼지농도(㎍/㎥)
0,2022-01-31 01:00,24
1,2022-01-31 02:00,26
2,2022-01-31 03:00,20
3,2022-01-31 04:00,28
4,2022-01-31 05:00,32
...,...,...
782,2022-03-05 18:00,36
783,2022-03-05 19:00,41
784,2022-03-05 21:00,33
785,2022-03-05 23:00,37


In [9]:
data = pd.merge(left = df , right = weather_df, how = "inner", on = "등록일자")
data = pd.merge(left = data , right = dust_df, how = "inner", on = "등록일자")

data.head(100)
data = data.drop_duplicates(['시리얼','등록일자'], keep='first')

data
#temp = df[df['시리얼']== 'V01T1622211']

,시리얼,관측시각,온도(℃),습도(%)_x,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자,기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위),1시간평균 미세먼지농도(㎍/㎥)
0,V10O1612855,2.022010e+11,0,41,65,24,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
2,V10O1612854,2.022010e+11,1,40,1,1,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
3,V10O1612850,2.022010e+11,0,42,38,14,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
4,V10O1612856,2.022010e+11,0,39,1,2,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
5,V10O1612853,2.022010e+11,0,41,30,11,-9999,-9999,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3397523,V10O1612861,2.022030e+11,3,31,27,9,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397524,V10O1612858,2.022030e+11,3,31,1,2,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397526,V01T1622211,2.022030e+11,21,15,23,17,129,318,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36
3397527,V10O1612860,2.022030e+11,2,33,1,2,-9999,-9999,2022-03-06 00:00,-0.6,3.4,43,0.0,0,36


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data["시리얼"] = encoder.fit_transform(data["시리얼"])

data = data.reset_index()


In [11]:
total_data = data[data.columns[2:]]
total_data = total_data.replace(-9999,np.nan)
total_data = total_data.fillna(method='bfill')
total_data = total_data.fillna(method='pad')
total_data.head(4)



,관측시각,온도(℃),습도(%)_x,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),등록일자,기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위),1시간평균 미세먼지농도(㎍/㎥)
0,2.022010e+11,0,41,65,24,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
1,2.022010e+11,1,40,1,1,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
2,2.022010e+11,0,42,38,14,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24
3,2.022010e+11,0,39,1,2,127.0,307.0,2022-01-31 01:00,-4.6,2.4,51,0.0,0,24


In [12]:
corr_df= total_data.corr(method='pearson')

corr_df


,관측시각,온도(℃),습도(%)_x,미세먼지(㎍/㎥),초미세먼지(㎍/㎥),휘발성유기화합물(ppb),이산화탄소(ppm),기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위),1시간평균 미세먼지농도(㎍/㎥)
관측시각,1.000000,-0.734735,-0.171863,0.024985,-0.103876,0.005540,-0.115155,0.001769,0.000990,-0.001452,0.001608,0.000881,0.000923
온도(℃),-0.734735,1.000000,0.121756,-0.034081,0.459017,0.013081,0.139355,0.037674,0.001999,0.002764,0.003315,0.015300,0.020584
습도(%)_x,-0.171863,0.121756,1.000000,0.043298,0.222512,0.048520,0.100891,0.012963,-0.024412,0.126223,-0.059686,0.042084,0.030257
미세먼지(㎍/㎥),0.024985,-0.034081,0.043298,1.000000,0.708439,0.003298,0.016845,0.004425,0.002786,0.000279,0.001425,0.001562,0.005196
초미세먼지(㎍/㎥),-0.103876,0.459017,0.222512,0.708439,1.000000,0.004048,0.061621,0.000793,0.000750,0.003106,-0.000605,0.000023,0.001752
휘발성유기화합물(ppb),0.005540,0.013081,0.048520,0.003298,0.004048,1.000000,0.214662,0.001476,-0.000686,0.000028,0.000496,0.000145,0.002521
이산화탄소(ppm),-0.115155,0.139355,0.100891,0.016845,0.061621,0.214662,1.000000,-0.001099,0.003838,-0.000243,0.001237,-0.001292,-0.000996
기온(°C),0.001769,0.037674,0.012963,0.004425,0.000793,0.001476,-0.001099,1.000000,0.164338,-0.033190,0.206940,0.427083,0.507584
풍속(m/s),0.000990,0.001999,-0.024412,0.002786,0.000750,-0.000686,0.003838,0.164338,1.000000,-0.271041,0.317957,-0.008569,0.041428
습도(%)_y,-0.001452,0.002764,0.126223,0.000279,0.003106,0.000028,-0.000243,-0.033190,-0.271041,1.000000,-0.514061,0.247748,0.193196


In [13]:
X =  total_data[total_data.columns[[1,2,5,6,8,9,10,11,12,13]]]
Y = total_data[total_data.columns[[3]]]


In [22]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=22)

scaler = MinMaxScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train, columns=X_train.columns, index=X_train.index)

X_scaled_test = scaler.fit_transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test, columns=X_test.columns, index=X_test.index)


X_scaled_train

,온도(℃),습도(%)_x,휘발성유기화합물(ppb),이산화탄소(ppm),기온(°C),풍속(m/s),습도(%)_y,일조(hr),전운량(10분위),1시간평균 미세먼지농도(㎍/㎥)
273649,0.851562,0.714286,0.019786,0.1224,0.672,0.371795,0.746479,0.0,0.4,0.781022
195491,0.945312,0.505495,0.004145,0.1416,0.248,0.384615,0.154930,0.0,0.0,0.000000
225171,0.953125,0.472527,0.013217,0.1712,0.372,0.102564,0.591549,0.6,0.0,0.313869
97754,0.921875,0.604396,0.021897,0.2268,0.364,0.217949,0.774648,1.0,0.0,0.562044
10426,0.945312,0.538462,0.013060,0.1708,0.352,0.282051,1.000000,0.0,0.9,0.029197
...,...,...,...,...,...,...,...,...,...,...
219638,0.968750,0.461538,0.001408,0.1372,0.748,0.628205,0.661972,0.0,1.0,0.189781
120166,0.953125,0.472527,0.001095,0.0832,0.524,0.141026,0.535211,0.0,0.0,0.233577
24932,0.921875,0.593407,0.008211,0.1788,0.296,0.166667,0.521127,0.0,0.8,0.087591
162752,0.953125,0.527473,0.003832,0.1968,0.180,0.641026,0.366197,0.0,0.0,0.153285


In [ ]:
# X_scaled_train = np.reshape(X_scaled_train, (2055210, 10 , 2))

In [ ]:
from keras.layers import *
from keras.models import *
from keras.utils import *


model = Sequential()
model.add(LSTM(256, input_dim=1, input_length=10, stateful=True))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



In [ ]:
# hist = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=1, validation_split=0.2)

In [29]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.4849285474477061

In [31]:
from sklearn.neural_network import MLPRegressor
model=MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000)
model.fit(X_scaled_train, y_train)
pred_train=model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)
pred_test=model.predict(X_scaled_test)



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련   데이터 RMSE: 12.350369723833708
테스트 데이터 RMSE: 12.019057408108528


In [32]:
model.score(X_scaled_test, y_test)

0.881147930059326

In [ ]:


import numpy as np
from sklearn.metrics import mean_squared_error 676166
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))


In [17]:
param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]}
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train_scaled, Y_train)
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_test_scaled, Y_test)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist

Best Parameter: {'n_neighbors': 11}
Best Score: 0.3607
TestSet Score: 0.3612
